<center>Jaime Francisco Barbosa Hernández   ID:615152<center>

<center>SC3314 – Inteligencia Artificial Universidad de Monterrey Dr. Antonio Martínez Torteya</center>

## A3.1 SVM y Multiple Testing

En esta actividad trabajarás con la base de datos de la que se habló en clase, que consiste de 83 muestras y 2308 variables de entrada, que consisten en la expresión génica estandarizada
de distintos genes. La variable de salida cuenta con valores numéricos del 1 al 4 que corresponden a distintos tipos de cáncer.

Desarrolla los siguientes puntos en una Jupyter Notebook, tratando, dentro de lo posible, que cada punto se trabaje en una celda distinta. Los comentarios en el código siempre son
bienvenidos, de preferencia, aprovecha el markdown para generar cuadros de descripción que ayuden al lector a comprender el trabajo realizado.


In [19]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import f_oneway
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

1. Importa los datos a tu ambiente de trabajo y revisa que no haya huecos. Calcula la diferencia de promedios entre las clases 2 y 4 para todos los genes, e imprime los 10 genes con la mayor diferencia de medias. Indica qué crees que esta diferencia podría implicar en términos de un estudio de inferencia.


In [10]:
df=pd.read_csv("A3.1 Khan.csv")

print("Dimension es del Data Frame: ",df.shape)
print()
print("Huecos: ",df.isnull().sum().sum())
print()
print(df.head())
print()

# Filtrar por clases 2 y 4
class_2 = df[df['y'] == 2].drop('y', axis=1)
class_4 = df[df['y'] == 4].drop('y', axis=1)

# Diferencia de medias
diff_means = class_2.mean() - class_4.mean()

# Top 10 genes con mayor diferencia absoluta
top10 = diff_means.abs().sort_values(ascending=False).head(10)
print(top10)

Dimension es del Data Frame:  (83, 2309)

Huecos:  0

         X1        X2        X3        X4        X5        X6        X7  \
0  0.773344 -2.438405 -0.482562 -2.721135 -1.217058  0.827809  1.342604   
1 -0.078178 -2.415754  0.412772 -2.825146 -0.626236  0.054488  1.429498   
2 -0.084469 -1.649739 -0.241308 -2.875286 -0.889405 -0.027474  1.159300   
3  0.965614 -2.380547  0.625297 -1.741256 -0.845366  0.949687  1.093801   
4  0.075664 -1.728785  0.852626  0.272695 -1.841370  0.327936  1.251219   

         X8        X9       X10  ...     X2300     X2301     X2302     X2303  \
0  0.057042  0.133569  0.565427  ... -0.027474 -1.660205  0.588231 -0.463624   
1 -0.120249  0.456792  0.159053  ... -0.246284 -0.836325 -0.571284  0.034788   
2  0.015676  0.191942  0.496585  ...  0.024985 -1.059872 -0.403767 -0.678653   
3  0.819736 -0.284620  0.994732  ...  0.357115 -1.893128  0.255107  0.163309   
4  0.771450  0.030917  0.278313  ...  0.061753 -2.273998 -0.039365  0.368801   

      X2304   

Una diferencia de medias entre clases 2 y 4 podría indicar que esos genes están expresados de forma distinta en los dos tipos de tejido o condición estudiada. En un contexto de inferencia, esto sugiere que esos genes podrían ser buenos candidatos para pruebas de hipótesis entre ambas clases, aunque aún se requeriría aplicar pruebas estadísticas más formales, como t-test o corrección por multiple testing para confirmar su verdadera significancia.

2. Calcula el estadístico t y el p-value para comparar las medias de todos los genes entre la clase 2 y la clase 4 de la base de datos. Usa la metodología de Bonferroni, de Holm, y de Benjamini-Hochberg para corregir por múltiples pruebas e indica, para cada una, qué genes tienen una expresión significativamente distinta entre las clases (maneja un control de 0.05). Te recomiendo usar la función multipletests de statsmodels.stats.multitest

In [12]:
# 3. Calcular t-test para cada gen
t_values = []
p_values = []

for gene in class_2.columns:
    t_stat, p_val = ttest_ind(class_2[gene], class_4[gene], equal_var=False)
    t_values.append(t_stat)
    p_values.append(p_val)

results = pd.DataFrame({
    'gene': class_2.columns,
    't_stat': t_values,
    'p_value': p_values
})

# 4. Correcciones por pruebas múltiples
alpha = 0.05

# Bonferroni
results['bonf_reject'], results['bonf_p'], _, _ = multipletests(results['p_value'], alpha=alpha, method='bonferroni')

# Holm
results['holm_reject'], results['holm_p'], _, _ = multipletests(results['p_value'], alpha=alpha, method='holm')

# Benjamini-Hochberg (FDR)
results['bh_reject'], results['bh_p'], _, _ = multipletests(results['p_value'], alpha=alpha, method='fdr_bh')

# 5. Genes significativos en cada método
sig_bonf = results[results['bonf_reject']]
sig_holm = results[results['holm_reject']]
sig_bh   = results[results['bh_reject']]

print("Significativos (Bonferroni):", sig_bonf.shape[0])
print("Significativos (Holm):", sig_holm.shape[0])
print("Significativos (Benjamini-Hochberg):", sig_bh.shape[0])

# Opcional: mostrar nombres de genes significativos
print("\nBonferroni genes:\n", sig_bonf['gene'].values)
print("\nHolm genes:\n", sig_holm['gene'].values)
print("\nBH genes:\n", sig_bh['gene'].values)

Significativos (Bonferroni): 72
Significativos (Holm): 72
Significativos (Benjamini-Hochberg): 296

Bonferroni genes:
 ['X2' 'X36' 'X67' 'X129' 'X174' 'X187' 'X188' 'X229' 'X246' 'X251' 'X338'
 'X348' 'X368' 'X372' 'X373' 'X380' 'X430' 'X433' 'X509' 'X545' 'X554'
 'X558' 'X566' 'X603' 'X655' 'X714' 'X762' 'X910' 'X951' 'X971' 'X1003'
 'X1021' 'X1023' 'X1055' 'X1070' 'X1093' 'X1105' 'X1110' 'X1112' 'X1132'
 'X1194' 'X1196' 'X1207' 'X1217' 'X1298' 'X1319' 'X1327' 'X1330' 'X1372'
 'X1389' 'X1416' 'X1610' 'X1626' 'X1634' 'X1645' 'X1706' 'X1708' 'X1723'
 'X1738' 'X1799' 'X1888' 'X1896' 'X1911' 'X1924' 'X1954' 'X1955' 'X1980'
 'X2046' 'X2050' 'X2115' 'X2146' 'X2247']

Holm genes:
 ['X2' 'X36' 'X67' 'X129' 'X174' 'X187' 'X188' 'X229' 'X246' 'X251' 'X338'
 'X348' 'X368' 'X372' 'X373' 'X380' 'X430' 'X433' 'X509' 'X545' 'X554'
 'X558' 'X566' 'X603' 'X655' 'X714' 'X762' 'X910' 'X951' 'X971' 'X1003'
 'X1021' 'X1023' 'X1055' 'X1070' 'X1093' 'X1105' 'X1110' 'X1112' 'X1132'
 'X1194' 'X1196' 'X1207' '

En este inciso podemos ver que primero se realiza la prueba t para comparar las medias de todos los genes entre la clase 2 y la clase 4 de la base de datos y encontrar si tienen una diferecia estadísticamente significativa. Por último, se utilizan métodos de validación multiple (Bonferroni, Holm y Benjamini-Hochberg "FDR"), imprimiendo los genes más significativos considerados por cada método.

3. Realiza un experimento similar, pero ahora comparando las medias de las 4 clases de la base de datos. Para lograrlo, en vez de trabajar con el estadístico t, te recomiendo realizar pruebas de análisis de varianza (ANOVA). Dicha prueba la puedes realizar con la función f_oneway de scipy.stats, pero revisa bien cómo se deben ingresar los datos a dicha función, necesitarás primero estratificarlos por clase.

In [23]:
# Verifica qué clases existen
print(df['y'].unique())

# 2. Separar los datos por clase
groups = [df[df['y'] == c].drop('y', axis=1) for c in sorted(df['y'].unique())]

# 3. ANOVA para cada gen
f_stats = []
p_values = []

for gene in groups[0].columns:
    data = [g[gene] for g in groups]   # una lista de vectores (uno por clase)
    f_stat, p_val = f_oneway(*data)
    f_stats.append(f_stat)
    p_values.append(p_val)

results_anova = pd.DataFrame({
    'gene': groups[0].columns,
    'F_stat': f_stats,
    'p_value': p_values
})

# 4. Genes con diferencia significativa (α=0.05)
significant_genes = results_anova[results_anova['p_value'] < 0.05]

print("Genes con diferencia significativa entre las 4 clases (ANOVA):", significant_genes.shape[0])

# 4. Correcciones múltiples
alpha = 0.05

results_anova['bonf_reject'], results_anova['bonf_p'], _, _ = multipletests(results_anova['p_value'], alpha=alpha, method='bonferroni')
results_anova['holm_reject'], results_anova['holm_p'], _, _ = multipletests(results_anova['p_value'], alpha=alpha, method='holm')
results_anova['bh_reject'],   results_anova['bh_p'],   _, _ = multipletests(results_anova['p_value'], alpha=alpha, method='fdr_bh')

# 5. Genes significativos según cada método
sig_bonf = results_anova[results_anova['bonf_reject']]
sig_holm = results_anova[results_anova['holm_reject']]
sig_bh   = results_anova[results_anova['bh_reject']]

print("Genes significativos (Bonferroni):", sig_bonf.shape[0])
print("Genes significativos (Holm):", sig_holm.shape[0])
print("Genes significativos (Benjamini-Hochberg):", sig_bh.shape[0])
print()

print(significant_genes.head(10))

print("\nBonferroni (primeros 10):\n", sig_bonf.head(10))
print("\nHolm (primeros 10):\n", sig_holm.head(10))
print("\nBenjamini–Hochberg (primeros 10):\n", sig_bh.head(10))

[2 4 3 1]
Genes con diferencia significativa entre las 4 clases (ANOVA): 1303
Genes significativos (Bonferroni): 404
Genes significativos (Holm): 412
Genes significativos (Benjamini-Hochberg): 1162

   gene     F_stat       p_value
0    X1  59.118264  3.839240e-20
1    X2  31.279175  1.977997e-13
2    X3  13.099869  5.004749e-07
8    X9   8.156223  8.500660e-05
11  X12   3.313432  2.418413e-02
16  X17  12.543986  8.650499e-07
20  X21   3.958019  1.103851e-02
21  X22   3.792741  1.348825e-02
22  X23   2.739617  4.882749e-02
25  X26   2.761909  4.751099e-02

Bonferroni (primeros 10):
    gene     F_stat       p_value  bonf_reject        bonf_p  holm_reject  \
0    X1  59.118264  3.839240e-20         True  8.860966e-17         True   
1    X2  31.279175  1.977997e-13         True  4.565218e-10         True   
2    X3  13.099869  5.004749e-07         True  1.155096e-03         True   
16  X17  12.543986  8.650499e-07         True  1.996535e-03         True   
28  X29  17.558086  7.937953e-

Para este inciso, se realizó un proceso muy similar al anterior. Primeramente se hizo un análisis ANOVA para comparar las medias entre las 4 clases, y definir cuales tienen una diferencia significativa. Y a partir de lo obtenido en el ANOVA, se utilizan las metodologías de corrección anteriormente utilizadas. Por último se despliegan la cantidad de variables significativas según cada método, y se imprimen las primeras 10 variables para cada una.

4. Separa los datos en entrenamiento y prueba, construye y entrena un modelo de SVM con un kernel lineal, con un kernel polinomial de orden 3, y con un kernel radial (puedes usar los parámetros que gustes, no necesitas optimizar con validación cruzada). Para evitar que el tiempo de procesamiento sea exagerado, puedes seleccionar solamente algunas variables, partiendo de los resultados que obtuviste en los puntos anteriores. Esta no es una práctica adecuada, pues estamos cayendo en una situación de fuga de datos. Lo ideal sería que la selección de características se basara solamente en experimentos realizados con los datos de entrenamiento. Pero, en este caso, obviaremos este detalle.

In [33]:
selected_genes = np.random.choice(sig_bonf['gene'], size=40, replace=False)
X = df[selected_genes]
y = df['y']

# 3. Separar en entrenamiento y prueba (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 4. Escalamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Modelos SVM

# Lineal
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train_scaled, y_train)

# Polinomial grado 3
svm_poly = SVC(kernel='poly', degree=3)
svm_poly.fit(X_train_scaled, y_train)

# RBF
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train_scaled, y_train)


SVC()

Para este inciso, como lo indica el problema, se omite el paso de selección de características, que si bien es lo correcto de realizar, por fines de capacidad computacional no se hará ese paso. En lugar de ello, se seleccionaron 40 características de manera "al azar" con la función *random.choice* de **numpy**. Después se separaron en train-test el DF con una razón de 70/30 respectivamente, y asegurandose se estratificar. Como paso intermedio, se utiliza *StandardScaler* pra estalar las características. Y por último se entrenan los 3 modelos con sus respectivos kernels. 

5. Calcula, para los 3 modelos, las métricas que consideres importantes para comparar los desempeños. Indica qué opinas sobre los resultados, especificando si crees que uno de los kernels es mejor para esta tarea específica.

In [37]:
y_pred_linear = svm_linear.predict(X_test_scaled)
y_pred_poly = svm_poly.predict(X_test_scaled)
y_pred_rbf = svm_rbf.predict(X_test_scaled)
acc_linear = accuracy_score(y_test, y_pred_linear)
acc_poly = accuracy_score(y_test, y_pred_poly)
acc_rbf = accuracy_score(y_test, y_pred_rbf)
prec_linear = precision_score(y_test, y_pred_linear, average='weighted')
prec_poly = precision_score(y_test, y_pred_poly, average='weighted')
prec_rbf = precision_score(y_test, y_pred_rbf, average='weighted')
rec_linear = recall_score(y_test, y_pred_linear, average='weighted')
rec_poly = recall_score(y_test, y_pred_poly, average='weighted')
rec_rbf = recall_score(y_test, y_pred_rbf, average='weighted')
f1_linear = f1_score(y_test, y_pred_linear, average='weighted')
f1_poly = f1_score(y_test, y_pred_poly, average='weighted')
f1_rbf = f1_score(y_test, y_pred_rbf, average='weighted')
print("Exactitud SVM Lineal: ", acc_linear)
print("Exactitud SVM Polinomial: ", acc_poly)
print("Exactitud SVM RBF: ", acc_rbf)
print("\nPrecisión SVM Lineal: ", prec_linear)
print("Precisión SVM Polinomial: ", prec_poly)
print("Precisión SVM RBF: ", prec_rbf)
print("\nRecall SVM Lineal: ", rec_linear)
print("Recall SVM Polinomial: ", rec_poly)
print("Recall SVM RBF: ", rec_rbf)
print("\nF1-Score SVM Lineal: ", f1_linear)
print("F1-Score SVM Polinomial: ", f1_poly)
print("F1-Score SVM RBF: ", f1_rbf)

Exactitud SVM Lineal:  1.0
Exactitud SVM Polinomial:  0.8
Exactitud SVM RBF:  0.96

Precisión SVM Lineal:  1.0
Precisión SVM Polinomial:  0.8714285714285714
Precisión SVM RBF:  0.9644444444444444

Recall SVM Lineal:  1.0
Recall SVM Polinomial:  0.8
Recall SVM RBF:  0.96

F1-Score SVM Lineal:  1.0
F1-Score SVM Polinomial:  0.7906915113871636
F1-Score SVM RBF:  0.9589542483660131


Para los 3 modelos entrenados, se decidió obtener las métrica de *Exactitud*, *Precisión*, *Recall*, y *F1-Score*. Podemos observar que en general, el que peor desempeño tuvo fue el SVM con Kernel polinomial, pues obtuvo la puntuación más baja en todas las métricas. Los que en teoría mejor desempeño tuvieron fueron el SVM Lineal y el Radial. Si se tuviera que elegir un ganador según las métricas, viendo éstas el ganador sería el SVM Lineal, pues obtuvo un 100% en todo. Sin embargo recordemos que no se hizo un proceso de selección de características correcto, por lo que nuestros modelos muy probablemente tienen fugas de datos.